# Notebook for result analysis

Here we will perform the analysis of the computed entities.
The measure computed will consist of Precision, Recall and micro-averaged F-score.

## Working directory

This Notebook is design to run in Google Colaboratory and as such the working directory is a folder inside Google Drive.

If you wish to use a local IPython instance make sure to update the working directory and leave out the step of connectind the Notebook to Google Drive.

In [7]:
from google.colab import drive
drive.mount("/content/drive/")

# Access your Drive data using folder '/content/drive/MyDrive'

# Set the working directory
workdir = "/content/drive/MyDrive/ml_ner/datasets/"

!ls -lah "$workdir"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
total 2.5M
drwx------ 3 root root 4.0K Jan  2 17:03 btc
drwx------ 2 root root 4.0K Dec 30 19:28 CoNLL03
drwx------ 2 root root 4.0K Jan  1 18:43 emtd
-rw------- 1 root root 2.4M Dec 30 19:30 entity-recognition-datasets-master.zip
drwx------ 3 root root 4.0K Jan  2 17:03 wikigold


## Object for collecting and calculating the summary

In [52]:
# Summary object
class SummaryDataset:
  def __init__(self, name):
    self.name = name
    self.ner_results = {}

  def get_info(self):
    return self.ner_results

  def print_info(self):
    print(f"Dataset: {self.name}")
    print(f"Number of result objects: {len(self.ner_results)}")
    for result in self.ner_results.values():
      print(result.name)

  def get_results(self):
    return self.ner_results.values()

  def add_result(self, key, result):
    self.ner_results[key] = result

  def get_result(self, key):
    return self.ner_results[key]

  # Visualizations
  def visualize_results(self):
    pass


# Result object
class NerResult:
  def __init__(self, name):
    self.name = name
    self.tp = 0
    self.fp = 0
    self.fn = 0
    self.precision = 0
    self. recall = 0
    self.fscore = 0
  
  def print_info(self):
    print(f"TP: {self.tp}")
    print(f"FP: {self.fp}")
    print(f"FN: {self.fn}")

  def add_tp(self):
    self.tp += 1

  def add_fp(self):
    self.fp += 1

  def add_fn(self):
    self.fn += 1

  def _calculate_precision(self):
    self.precision = self.tp / (self.tp + self.fp)

  def _calculate_recall(self):
    self.recall = self.tp / (self.tp + self.fn)

  def _calculate_fscore(self):
    self.fscore = 2 * ((self.precision * self.recall) / (self.precision + self.recall))

  def calculate_summary(self):
    self._calculate_precision()
    self._calculate_recall()
    self._calculate_fscore()

    # Print on finish
    self.print_summary()

  def print_summary(self):
    print(f"NER System: {self.name}")
    print(f"Precision: {self.precision:.3}")
    print(f"Recall: {self.recall:.3}")
    print(f"F-score: {self.fscore:.3}")

## Read in and parse

Read in all the annotated data and parse it into a unified format for evaluation.

In [3]:
# imports
import os
import shutil
import json

from pathlib import PurePath

In [4]:
# Read in the data
def decode_data(file):
  # Read the file
  with open(file) as f:
    dataset = json.loads(f.read())
    
    return dataset

# Write out the data
def encode_data(path, filename, output):
  # Export the json file
  json_dump = json.dumps(output)

  with open(PurePath(path, filename), "w") as f:
    f.write(json_dump)

## Analysis computation

Taking the results from NER and computing the performance

In [32]:
# Mapper for the entities that are not included in the datasets

ent_mapper_wikigold = {
    "I-PER": ["I-PER", "I-PERSON", "B-PER", "B-PERSON", "E-PERSON", "S-PERSON"],
    "I-LOC": ["I-LOC", "I-LOCATION", "I-GPE", "I-GSP", "B-LOC", "B-LOCATION", "B-GPE", "B-GSP", "S-LOC", "S-GPE", "E-LOC", "E-GPE"],
    "I-ORG": ["I-FAC", "I-FACILITY", "I-NORP", "I-ORG", "I-ORGANIZATION", "B-FAC", "B-FACILITY", "B-ORGANIZATION", "B-NORP", 
              "B-ORG","S-FAC", "S-NORP", "S-ORG","E-FAC", "E-NORP", "E-ORG"],
    "I-MISC": ["I-CARDINAL", "I-DATE", "I-EVENT", "I-LANGUAGE", "I-LAW", "I-MONEY",
               "I-ORDINAL", "I-PERCENT", "I-PRODUCT", "I-QUANTITY", "I-TIME", "I-WORK_OF_ART",
               "B-CARDINAL", "B-DATE", "B-EVENT", "B-LANGUAGE", "B-LAW", "B-MONEY",
               "B-ORDINAL", "B-PERCENT", "B-PRODUCT", "B-QUANTITY", "B-TIME", "B-WORK_OF_ART",
               "S-CARDINAL", "S-DATE", "S-EVENT", "S-LANGUAGE", "S-LAW", "S-MONEY",
               "S-ORDINAL", "S-PERCENT", "S-PRODUCT", "S-QUANTITY", "S-TIME", "S-WORK_OF_ART",
               "E-CARDINAL", "E-DATE", "E-EVENT", "E-LANGUAGE", "E-LAW", "E-MONEY",
               "E-ORDINAL", "E-PERCENT", "E-PRODUCT", "E-QUANTITY", "E-TIME", "E-WORK_OF_ART"
              ],
    "O": ["O"]
}

ent_mapper_wikigold_2 = {
    "I-PER": ["I-PER", "I-PERSON", "B-PER", "B-PERSON", "E-PERSON", "S-PERSON"],
    "I-LOC": ["I-LOC", "I-LOCATION", "B-LOC", "B-LOCATION", "S-LOC", "E-LOC"],
    "I-ORG": ["I-ORG", "I-ORGANIZATION", "B-ORGANIZATION", 
              "B-ORG", "S-ORG", "E-ORG"],
    "I-MISC": ["I-CARDINAL", "I-DATE", "I-EVENT", "I-LANGUAGE", "I-LAW", "I-MONEY",
               "I-ORDINAL", "I-PERCENT", "I-PRODUCT", "I-QUANTITY", "I-TIME", "I-WORK_OF_ART",
               "B-CARDINAL", "B-DATE", "B-EVENT", "B-LANGUAGE", "B-LAW", "B-MONEY",
               "B-ORDINAL", "B-PERCENT", "B-PRODUCT", "B-QUANTITY", "B-TIME", "B-WORK_OF_ART",
               "S-CARDINAL", "S-DATE", "S-EVENT", "S-LANGUAGE", "S-LAW", "S-MONEY",
               "S-ORDINAL", "S-PERCENT", "S-PRODUCT", "S-QUANTITY", "S-TIME", "S-WORK_OF_ART",
               "E-CARDINAL", "E-DATE", "E-EVENT", "E-LANGUAGE", "E-LAW", "E-MONEY",
               "E-ORDINAL", "E-PERCENT", "E-PRODUCT", "E-QUANTITY", "E-TIME", "E-WORK_OF_ART",
               "I-GPE", "I-GSP", "B-GPE", "B-GSP", "S-GPE", "E-GPE",
               "I-FAC", "I-FACILITY", "I-NORP", "B-FAC", "B-FACILITY", "B-NORP",
               "S-FAC", "S-NORP", "E-FAC", "E-NORP"
              ],
    "O": ["O"]
}

ent_mapper_btc = {
    "I-PER": ["I-PER", "I-PERSON", "E-PERSON"],
    "B-PER": ["B-PER", "B-PERSON", "S-PERSON"],
    "I-LOC": ["I-LOC", "I-LOCATION", "I-GPE", "I-GSP", "E-LOC", "E-GPE"],
    "B-LOC": ["B-LOC", "B-LOCATION", "B-GPE", "B-GSP", "S-LOC", "S-GPE"],
    "I-ORG": ["I-FAC", "I-FACILITY", "I-NORP", "I-ORG", "I-ORGANIZATION", "E-FAC", "E-NORP", "E-ORG"],
    "B-ORG": ["B-FAC", "B-FACILITY", "B-ORGANIZATION", "B-NORP", "B-ORG","S-FAC", "S-NORP", "S-ORG"],
    "O": ["O", "I-CARDINAL", "I-DATE", "I-EVENT", "I-LANGUAGE", "I-LAW", "I-MONEY",
          "I-ORDINAL", "I-PERCENT", "I-PRODUCT", "I-QUANTITY", "I-TIME", "I-WORK_OF_ART",
          "B-CARDINAL", "B-DATE", "B-EVENT", "B-LANGUAGE", "B-LAW", "B-MONEY",
          "B-ORDINAL", "B-PERCENT", "B-PRODUCT", "B-QUANTITY", "B-TIME", "B-WORK_OF_ART",
          "S-CARDINAL", "S-DATE", "S-EVENT", "S-LANGUAGE", "S-LAW", "S-MONEY",
          "S-ORDINAL", "S-PERCENT", "S-PRODUCT", "S-QUANTITY", "S-TIME", "S-WORK_OF_ART",
          "E-CARDINAL", "E-DATE", "E-EVENT", "E-LANGUAGE", "E-LAW", "E-MONEY",
          "E-ORDINAL", "E-PERCENT", "E-PRODUCT", "E-QUANTITY", "E-TIME", "E-WORK_OF_ART"
         ]
}

ent_mapper_btc_2 = {
    "I-PER": ["I-PER", "I-PERSON", "E-PERSON"],
    "B-PER": ["B-PER", "B-PERSON", "S-PERSON"],
    "I-LOC": ["I-LOC", "I-LOCATION", "E-LOC"],
    "B-LOC": ["B-LOC", "B-LOCATION", "S-LOC"],
    "I-ORG": ["I-ORG", "I-ORGANIZATION", "E-ORG"],
    "B-ORG": ["B-ORGANIZATION", "B-ORG", "S-ORG"],
    "O": ["O", "I-CARDINAL", "I-DATE", "I-EVENT", "I-LANGUAGE", "I-LAW", "I-MONEY",
          "I-ORDINAL", "I-PERCENT", "I-PRODUCT", "I-QUANTITY", "I-TIME", "I-WORK_OF_ART",
          "B-CARDINAL", "B-DATE", "B-EVENT", "B-LANGUAGE", "B-LAW", "B-MONEY",
          "B-ORDINAL", "B-PERCENT", "B-PRODUCT", "B-QUANTITY", "B-TIME", "B-WORK_OF_ART",
          "S-CARDINAL", "S-DATE", "S-EVENT", "S-LANGUAGE", "S-LAW", "S-MONEY",
          "S-ORDINAL", "S-PERCENT", "S-PRODUCT", "S-QUANTITY", "S-TIME", "S-WORK_OF_ART",
          "E-CARDINAL", "E-DATE", "E-EVENT", "E-LANGUAGE", "E-LAW", "E-MONEY",
          "E-ORDINAL", "E-PERCENT", "E-PRODUCT", "E-QUANTITY", "E-TIME", "E-WORK_OF_ART",
          "I-GPE", "I-GSP", "E-GPE", "B-GPE", "B-GSP", "S-GPE", "I-FAC", "I-FACILITY", "I-NORP",
          "E-FAC", "E-NORP", "B-FAC", "B-FACILITY", "B-NORP", "S-FAC", "S-NORP"
         ]
}

In [56]:
# List of NER systems in data
ner_systems = ["spaCy", "Stanza", "Classla", "NLTK"]

# Set filepaths for read in
filepaths = [PurePath(workdir, "wikigold", "results"), PurePath(workdir, "btc", "results")]

# List to store the summary data
summaries = []

# Process all the files
for filepath in filepaths:
  for path, _, files in os.walk(PurePath(filepath)):
    for name in files:
      # Compile the absolute filepath
      file = PurePath(path, name)

      # Select the appropriate mapper
      mapper = ent_mapper_wikigold_2
      if "btc" in PurePath(file).parts:
        mapper = ent_mapper_btc_2

      # Read in the dataset
      dataset = decode_data(file)

      # Create the SummaryDataset object
      summary = SummaryDataset(str(file))

      # Create the NerResult objects
      for ner_system in ner_systems:
        summary.add_result(ner_system, NerResult(ner_system))

      # Iterate over the sentences in the dataset
      for entry in dataset:
        # Check tp, fp, fn for every system
        # Go over annotations and check if matching with systems
        for annotation, spacy_entity, stanza_entity, classla_entity, nltk_entity in zip(entry["annotations"], entry["spacy_entities"], entry["stanza_entities"], entry["classla_entities"], entry["nltk_entities"]):
          # spaCy
          if spacy_entity in mapper[annotation]:
            summary.get_result("spaCy").add_tp()
          elif spacy_entity != "O":
            summary.get_result("spaCy").add_fp()
          else:
            summary.get_result("spaCy").add_fn()
          
          # Stanza
          if stanza_entity in mapper[annotation]:
            summary.get_result("Stanza").add_tp()
          elif stanza_entity != "O":
            summary.get_result("Stanza").add_fp()
          else:
            summary.get_result("Stanza").add_fn()
          
          # Classla
          if classla_entity in mapper[annotation]:
            summary.get_result("Classla").add_tp()
          elif classla_entity != "O":
            summary.get_result("Classla").add_fp()
          else:
            summary.get_result("Classla").add_fn()
        
          # NLTK
          if nltk_entity in mapper[annotation]:
            summary.get_result("NLTK").add_tp()
          elif nltk_entity != "O":
            summary.get_result("NLTK").add_fp()
          else:
            summary.get_result("NLTK").add_fn()

      #summary.get_result("spaCy").print_info()
      #summary.get_result("Stanza").print_info()
      #summary.get_result("Classla").print_info()
      #summary.get_result("NLTK").print_info()

      # Calculate and print metrics
      print(f"{'':_>150}")
      summary.print_info()
      for result in summary.get_results():
        print(f"{'':->50}")
        result.calculate_summary()
        print(f"{'':->50}")
      print(f"{'':_>150}\n")




______________________________________________________________________________________________________________________________________________________
Dataset: /content/drive/MyDrive/ml_ner/datasets/wikigold/results/wikigold_results.json
Number of result objects: 4
spaCy
Stanza
Classla
NLTK
--------------------------------------------------
NER System: spaCy
Precision: 0.857
Recall: 0.978
F-score: 0.913
--------------------------------------------------
--------------------------------------------------
NER System: Stanza
Precision: 0.842
Recall: 0.981
F-score: 0.906
--------------------------------------------------
--------------------------------------------------
NER System: Classla
Precision: 0.784
Recall: 0.89
F-score: 0.834
--------------------------------------------------
--------------------------------------------------
NER System: NLTK
Precision: 0.92
Recall: 0.954
F-score: 0.937
--------------------------------------------------
____________________________________________

## Flush changes

If ran in Google Colaboratory

In [57]:
# Run this at the end

drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
